In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import torch 
from torch import nn 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn import metrics

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

## Problem 3

### Make the NN model

In [4]:
data_path = 'data_assignment_1.csv'

#Data import
df3 = pd.read_csv(data_path)

In [5]:
df3.iloc[:, 3:]

,Frontal_Sup,Frontal_Inf,Cingulum_Ant,Cingulum_Post,Parietal_Sup,Parietal_Inf,Occipital_Sup,Occipital_Inf,Temporal_Sup,Temporal_Inf
0,0.541704,0.553985,0.577727,0.502631,0.539654,0.562739,0.584094,0.656325,0.529551,0.521926
1,0.665915,0.477778,0.525422,0.473192,0.589977,0.656130,0.677115,0.571440,0.430463,0.499023
2,0.385500,0.535842,0.651637,0.556026,0.451074,0.602841,0.578349,0.626247,0.511696,0.531485
3,0.601210,0.500756,0.614601,0.581634,0.489078,0.615674,0.579836,0.548231,0.499937,0.612477
4,0.529871,0.521045,0.576609,0.557342,0.494459,0.577652,0.668027,0.520755,0.521910,0.503761
...,...,...,...,...,...,...,...,...,...,...
95,0.555299,0.597638,0.553309,0.523025,0.525431,0.667517,0.502397,0.601756,0.494372,0.637729
96,0.524961,0.631617,0.382947,0.410978,0.505315,0.611400,0.605452,0.625818,0.445138,0.534949
97,0.490657,0.652892,0.654747,0.656929,0.521088,0.702478,0.640216,0.537376,0.642808,0.598981
98,0.584763,0.602108,0.465685,0.602878,0.693304,0.707188,0.685981,0.683557,0.697648,0.688851


In [6]:
df3.iloc[:, 2]

0     0
1     0
2     0
3     0
4     0
     ..
95    1
96    1
97    1
98    1
99    1
Name: diagnosis, Length: 100, dtype: int64

In [7]:
#Shuffle the data
df3_s = df3.sample(frac = 1)

In [8]:
#k_fold preparation
k = 5
data_lst = []
indices_lst = []

for i in range(k):
    indices_lst.append(int(len(df3_s)/k)*i)
indices_lst.append(len(df3_s))


for j in range(k):
    #Obtained train_test splitted dataframe
    split_test = df3_s[indices_lst[j]:indices_lst[j+1]]
    split_train = df3_s.drop(split_test.index)
    
    train_X = split_train.iloc[:, 3:].values
    train_y = split_train.iloc[:, 2].values
    
    test_X = split_test.iloc[:, 3:].values
    test_y = split_test.iloc[:, 2].values
    
    data_lst.append([[train_X,train_y],[test_X,test_y]])

In [9]:
data_lst[0][0]

[array([[0.71094965, 0.6071414 , 0.66616837, 0.54308243, 0.59673554,
         0.51014486, 0.64550388, 0.5516656 , 0.5172243 , 0.52235017],
        [0.7525998 , 0.62754802, 0.71992755, 0.67317934, 0.70290128,
         0.57679997, 0.64177104, 0.5844612 , 0.6817547 , 0.60331811],
        [0.57094409, 0.50026239, 0.48965384, 0.55105441, 0.59281946,
         0.59672913, 0.49102359, 0.5802805 , 0.50105927, 0.55973703],
        [0.55456018, 0.48502469, 0.44348991, 0.5748706 , 0.6701807 ,
         0.64780626, 0.60076257, 0.57699046, 0.3418852 , 0.57218686],
        [0.5563761 , 0.41146176, 0.55495914, 0.45852382, 0.47891037,
         0.65847702, 0.46100798, 0.45092439, 0.55412222, 0.55897064],
        [0.59687853, 0.69920276, 0.56772984, 0.66228864, 0.61587568,
         0.66004142, 0.67895774, 0.58710225, 0.60429313, 0.55923498],
        [0.51837613, 0.58830442, 0.54578292, 0.59432137, 0.51511336,
         0.62127787, 0.56080146, 0.6181794 , 0.58891435, 0.51755801],
        [0.61290346, 0.6362

In [10]:
#Create Model, referenced tutorial
class NNClassifier(nn.Module):
    def __init__(self, input_size = None):
        super().__init__()  
        layers = []
        #input
        layers.append(nn.Linear(input_size, 8))
        layers.append(nn.ReLU())
        #hidden
        layers.append(nn.Linear(8,6))
        layers.append(nn.ReLU())
        #output
        layers.append(nn.Linear(6, 2))

        self.layers = nn.Sequential(*layers)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
    
    def predict(self, X):
        self.eval()
        with torch.no_grad():
            pred = self(X)
            return int(pred.argmax().detach())     
        

In [11]:
input_len = 10
model = NNClassifier(input_len)
print(model)

NNClassifier(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=6, bias=True)
    (3): ReLU()
    (4): Linear(in_features=6, out_features=2, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax): Softmax(dim=1)
)


In [18]:
# Referenced from tutorial
def train_model(model, train_dl, optimizer, loss_f, nr_epochs, print_loss_every=10):
    for t in range(nr_epochs):
        model.train()
        nr_batches = len(train_dl)
        total_loss = 0
        optimizer.zero_grad()
        
        
        pred_lst = []
        target_lst = []
        
        for _, (X, y) in enumerate(train_dl):
            pred = model(X)
            loss = loss_f(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            pred_lst.append(pred.detach().numpy())
            target_lst.append(y.detach().numpy())
        
        pred_lst = np.concatenate(pred_lst, axis=0)
        target_lst =  np.concatenate(target_lst, axis=0)

        
        if t % print_loss_every == 0:
            print(f"Epoch {t} loss {total_loss / nr_batches}, accuracy {metrics.accuracy_score(target_lst, pred_lst.argmax(1))}")
            
    print("Training complete")
    
def eval_model(model, dl):
    model.eval()
    targets = []
    predictions = []
    for _, (X, y) in enumerate(dl):
        pred = model(X)
        targets += list(y.detach().cpu().numpy())
        predictions += list(pred.argmax(1).detach().cpu().numpy())
        
    accuracy = metrics.accuracy_score(targets, predictions)
    precision = metrics.precision_score(targets,predictions)
    recall = metrics.recall_score(targets,predictions)
    fpr, tpr, _ = metrics.roc_curve(targets, predictions, pos_label=1)
    auc = metrics.auc(fpr,tpr)
    print('Accuracy: '+ str(accuracy))
    print('Precision: '+ str(precision))
    print('Recall: ' + str(recall))
    print('AUC: ' + str(auc))
    
    return [accuracy,precision,recall,auc]

In [19]:
class df2Dataset(Dataset):
    def __init__(self, df_lst, y_dtype=torch.int64):
        self.X = torch.tensor(df_lst[0],dtype=torch.float32)
        self.y = torch.tensor(df_lst[1],dtype=torch.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [20]:
#Train model and evalute
score_lst = []
fold_cnt = 1
no_epoch = 100


for i in data_lst:
    train_dl = DataLoader(df2Dataset(i[0])) 
    test_dl = DataLoader(df2Dataset(i[1]))
    print("Current fold: "+ str(fold_cnt))
    fold_cnt+=1
    
    model = NNClassifier(10) 
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
    loss_function = nn.CrossEntropyLoss()
    
    train_model(model, train_dl, optimizer, loss_function,no_epoch, )
    score_lst.append(eval_model(model,test_dl))
    

Current fold: 1
Epoch 0 loss 0.6959626384079456, accuracy 0.5
Epoch 10 loss 0.693686268478632, accuracy 0.4625
Epoch 20 loss 0.6928190261125564, accuracy 0.5375
Epoch 30 loss 0.6898744039237499, accuracy 0.6
Epoch 40 loss 0.6777487851679325, accuracy 0.625
Epoch 50 loss 0.6374081518501044, accuracy 0.675
Epoch 60 loss 0.5984674904495477, accuracy 0.675
Epoch 70 loss 0.5775482771918178, accuracy 0.6625
Epoch 80 loss 0.5655128189362586, accuracy 0.6625
Epoch 90 loss 0.5588103746064007, accuracy 0.6625
Training complete
Accuracy: 0.55
Precision: 0.6
Recall: 0.3
AUC: 0.55
Current fold: 2
Epoch 0 loss 0.7166630454361439, accuracy 0.4875
Epoch 10 loss 0.6908059202134609, accuracy 0.5125
Epoch 20 loss 0.6875861465930939, accuracy 0.575
Epoch 30 loss 0.681240015476942, accuracy 0.6625
Epoch 40 loss 0.6679650418460369, accuracy 0.7
Epoch 50 loss 0.6446383655071258, accuracy 0.6625
Epoch 60 loss 0.6129213776439428, accuracy 0.6875
Epoch 70 loss 0.5813908971846103, accuracy 0.7125
Epoch 80 loss 0

### Log Regression and SVM

In [26]:
data_lst[0][1][1]

array([0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1],
      dtype=int64)

In [29]:
svm_score_lst = []
log_score_lst = []

fold_cnt = 1
for i in data_lst:
    print("Current fold: "+ str(fold_cnt))
    fold_cnt+=1
    
    X_train = i[0][0]
    Y_train = i[0][1]
    X_test = i[1][0]
    Y_test = i[1][1]
    
    
    log_model = LogisticRegression(random_state=0, max_iter=200)
    log_model.fit(X_train, Y_train)

    logtrain_pred = log_model.predict(X_train)
    logtest_pred = log_model.predict(X_test)
    
    
    accuracy = metrics.accuracy_score(Y_test,logtest_pred)
    precision = metrics.precision_score(Y_test,logtest_pred)
    recall = metrics.recall_score(Y_test,logtest_pred)
    fpr, tpr, _ = metrics.roc_curve(Y_test,logtest_pred, pos_label=1)
    auc = metrics.auc(fpr,tpr)
    print("log test accuracy: ", accuracy)

    log_score_lst.append([accuracy,precision,recall,auc])
    
    svm = SVC(gamma='auto', kernel='linear')
    svm.fit(X_train, Y_train)
    svmtrain_pred = svm.predict(X_train)
    svmtest_pred = svm.predict(X_test)
    
    accuracy = metrics.accuracy_score(Y_test,svmtest_pred)
    precision = metrics.precision_score(Y_test,svmtest_pred)
    recall = metrics.recall_score(Y_test,svmtest_pred)
    fpr, tpr, _ = metrics.roc_curve(Y_test,svmtest_pred, pos_label=1)
    auc = metrics.auc(fpr,tpr)
    print("svm test accuracy: ", accuracy)
    
    log_score_lst.append([accuracy,precision,recall,auc])
    

Current fold: 1
log test accuracy:  0.8
svm test accuracy:  0.7
Current fold: 2
log test accuracy:  0.55
svm test accuracy:  0.55
Current fold: 3
log test accuracy:  0.85
svm test accuracy:  0.55
Current fold: 4
log test accuracy:  0.55
svm test accuracy:  0.4
Current fold: 5
log test accuracy:  0.65
svm test accuracy:  0.5


# Problem 4

In [ ]:
data_path_4 = ''